In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
sc = (
    SparkSession
    .builder
    .master("local[2]")
    .appName("hw05")
    .getOrCreate()
)


In [4]:
df = sc.read.format("csv").load("file:///home/jovyan/work/train.csv")


In [5]:
df.show(10)

+--------------------+--------------------+--------------------+------------+-------+------+------+-------------+
|                 _c0|                 _c1|                 _c2|         _c3|    _c4|   _c5|   _c6|          _c7|
+--------------------+--------------------+--------------------+------------+-------+------+------+-------------+
|                  id|        comment_text|               toxic|severe_toxic|obscene|threat|insult|identity_hate|
|    0000997932d777bf|         Explanation|                null|        null|   null|  null|  null|         null|
|Why the edits mad...| just closure on ...|                   0|           0|      0|     0|     0|            0|
|    000103f0d9cfb60f|D'aww! He matches...|                   0|           0|      0|     0|     0|            0|
|    000113f07ec002fd|Hey man, I'm real...|                   0|           0|      0|     0|     0|            0|
|    0001b41b1c6bb37e|                   "|                null|        null|   null|  n